### Работа с ollama моделями через LangChain

In [1]:
import numpy as np
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings

In [2]:
llm = OllamaLLM(model="llama3.2", temperature=1)
response = llm.invoke("hi, do u love cats")
print(response)

Hello! I don't have personal feelings or emotions, but I can certainly chat with you about cats! Many people adore cats for their adorable antics, playful personalities, and affectionate nature. What's your favorite thing about cats?


In [3]:
embeddings = OllamaEmbeddings(model="all-minilm:33m")

texts = [
    "Что такое LangChain?",
    "Как использовать Ollama для создания эмбеддингов?",
    "Какие модели поддерживает Ollama?"
]

vectors = embeddings.embed_documents(texts)

vectors = np.array(vectors)
new_matr = vectors/np.linalg.norm(vectors, axis=1, keepdims=True)
new_matr @ new_matr.T

array([[1.        , 0.19043786, 0.27984475],
       [0.19043786, 1.        , 0.68144236],
       [0.27984475, 0.68144236, 1.        ]])

### Примеры для Weaviate

In [4]:
import weaviate
from weaviate.classes.init import Auth
from weaviate.classes.config import Configure
from weaviate.classes.query import MetadataQuery
import requests, json, os

#### Инициализация клиента

In [5]:
client = weaviate.connect_to_local()
print(client.is_ready())

True


#### Коллекции

In [ ]:
# Удалить все коллекции
client.collections.delete_all()

In [ ]:
# Создать коллекцию с привязанным векторизатором. Два варианта векторизаторов
questions = client.collections.create(
    name="Question",
    #vectorizer_config=Configure.Vectorizer.text2vec_ollama(     # Configure the Ollama embedding integration
    #    api_endpoint="http://ollama:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
    #    model="all-minilm:33m",                              # The model to use
    #),
    vectorizer_config =[
        Configure.NamedVectors.text2vec_ollama(
            name="title_vector",
            source_properties=["answer"],
            api_endpoint="http://ollama:11434",  # If using Docker, use this to contact your local Ollama instance
            model="all-minilm:33m",  # The model to use, e.g. "nomic-embed-text"
    )
    ],
    generative_config=Configure.Generative.ollama(              # Configure the Ollama generative integration
        api_endpoint="http://ollama:11434",       # Allow Weaviate from within a Docker container to contact your Ollama instance
        model="llama3.2",                                       # The model to use
    )
)

In [ ]:
# Наполнить коллекцию
resp = requests.get(
    "https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json"
)
data = json.loads(resp.text)

questions = client.collections.get("Question")

with questions.batch.dynamic() as batch:
    for d in data:
        batch.add_object({
            "answer": d["Answer"],
            "question": d["Question"],
            "category": d["Category"],
        })

In [8]:
# Провести векторный поиск по полю answer
questions = client.collections.get("Question")

response = questions.query.near_text(
    query="Elephant",
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

{
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
}
{
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
}


In [ ]:
# Провести векторный поиск и поиск по ключевым словам по полю answer
questions = client.collections.get("Question")

response = questions.query.hybrid(
    query="Elephant",  # The model provider integration will automatically vectorize the query
    limit=2
)

for obj in response.objects:
    print(json.dumps(obj.properties, indent=2))

{
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
}
{
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
}


In [ ]:
# Найти документ, наиболее близкий по полю "answer" к query, и составить на его основе твит с помощью llama3.2
questions = client.collections.get("Question")

response = questions.generate.near_text(
    query="Elephant",
    limit=2,
    grouped_task="Write a tweet with emojis about these facts."
)

print(response.generated)  # Inspect the generated text

"🐘 Elephants are the only living mammals in the Proboseidea order! 🤔 Did you know their distinctive 👃 noses set them apart from other reptiles like the gavial? 🦈 #ElephantFacts #WildlifeWonders"


In [18]:
client.close()